# Densenet

In this week's assignment, you'll be using a pre-trained Densenet model for image classification.

Densenet is a convolutional network where each layer is connected to all other layers that are deeper in the network
- The first layer is connected to the 3rd, 4th etc.
- The second layer is connected to the 3rd, 4th, 5th etc.

Like this:

<img src="images/densenet.png" alt="U-net Image" width="400" align="middle"/>

For a detailed explanation of Densenet, check out the source of the image above, a paper by Gao Huang et al. 2018 called [Densely Connected Convolutional Networks](https://arxiv.org/pdf/1608.06993.pdf).

The cells below are set up to provide an exploration of the Keras densenet implementation that you'll be using in the assignment. Run these cells to gain some insight into the network architecture.

In [1]:
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

For your work in the assignment, you'll be loading a set of pre-trained weights to reduce training time.

In [2]:
# create the base pre-trained model
base_model = DenseNet121(weights='models/densenet.hdf5', include_top=False)

In [4]:
# view summary model
print(base_model.summary())

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, None, None,   0          ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1/conv (Conv2D)            (None, None, None,   9408        ['zero_padding2d[0][0]']         
                                64)                                                     

In [5]:
# print first 5 layers
layers = base_model.layers

layers[:5]

In [6]:
# print last 5
layers[-5:]

In [7]:
# get conv layers and print the first 5
conv2d_layers = [layer for layer in base_model.layers if str(type(layer)).find('Conv2D') > -1]
conv2d_layers

In [8]:
# print the number of channels in the input
base_model.input

<KerasTensor: shape=(None, None, None, 3) dtype=float32 (created by layer 'input_1')>

In [9]:
# get the number of output channels
x = base_model.output

print(x)

# add global spatial average pooling layer
x_pool = GlobalAveragePooling2D()(x)

KerasTensor(type_spec=TensorSpec(shape=(None, None, None, 1024), dtype=tf.float32, name=None), name='relu/Relu:0', description="created by layer 'relu'")


In [10]:
# define a set of five class labels to use as an example
labels = ['Emphysema',
          'Hernia',
          'Mass',
          'Pneumonia',
          'Edema']
n_classes = len(labels)


In [12]:
# add logistic layer the same size as the number of classes
predictions = Dense(n_classes, activation='sigmoid')(x_pool)
print(f"Predictions have {n_classes} units, one for each class")
predictions

Predictions have 5 units, one for each class


<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'dense_1')>

In [13]:
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy')